__Задача 1__

Дана матрица объект-признак
```
X = [[ 1.22401313,  2.30868478,  3.03636353,  2.69287214],
     [-0.18757272,  1.30337355,  5.12093014,  3.46363202],
     [-0.81094525,  1.82463398,  5.79686488,  1.86159445],
     [ 0.75129018,  2.67392052,  3.65529809,  1.66746094],
     [ 0.00972362,  1.97367255,  2.50594319,  1.69755173],
     [-0.62972637,  0.77750764,  2.84124027,  4.54410559],
     [ 2.29536229,  1.81206697,  1.95026215,  1.51874636],
     [ 0.0920418 ,  2.26971361,  7.47708735,  2.61081203],
     [ 2.39252799,  3.17563985,  3.61420599,  5.10773362],
     [ 0.54983815,  2.87988651,  1.65752765,  1.59635987]]
```
и значения целевой переменной
```
y = [ 9.26193358,  9.700363  ,  8.67214805,  8.74796974,  6.18689108,
      7.53312713,  7.57643777, 12.44965478, 14.29010746,  6.68361218]
```

Подберите два признака (из четырёх) так, чтобы уровень линейной зависимости целевой переменной от значений этих признаков был максимальным. Другими словами, модель линейной регрессии на этих признаках должна давать наилучший результат. В качестве ответа запишите значение коэффициента детерминации для модели на этих признаках.

__Решение__

Найдем коэффициенты линейной зависимости целевой переменной от каждого из признаков: 

In [1]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression

In [2]:
x = pd.DataFrame([[1.22401313,  2.30868478,  3.03636353,  2.69287214],
     [-0.18757272,  1.30337355,  5.12093014,  3.46363202],
     [-0.81094525,  1.82463398,  5.79686488,  1.86159445],
     [ 0.75129018,  2.67392052,  3.65529809,  1.66746094],
     [ 0.00972362,  1.97367255,  2.50594319,  1.69755173],
     [-0.62972637,  0.77750764,  2.84124027,  4.54410559],
     [ 2.29536229,  1.81206697,  1.95026215,  1.51874636],
     [ 0.0920418 ,  2.26971361,  7.47708735,  2.61081203],
     [ 2.39252799,  3.17563985,  3.61420599,  5.10773362],
     [ 0.54983815,  2.87988651,  1.65752765,  1.59635987]])


In [3]:
y = np.array([9.26193358,  9.700363,  8.67214805,  8.74796974,  6.18689108, 7.53312713,  7.57643777, 12.44965478, 14.29010746,  6.68361218])

In [4]:
y.shape

(10,)

In [5]:
for i in range(0, 4):
  b1 = np.cov(x[i], y, ddof=1)[0, 1] / np.var(x[i], ddof=1)
  print(b1)

0.7991693729496844
1.4003156134853512
0.803310579194931
1.1783848740628293


Из расчетов делаем вывод, что максимально влияют на целевую перменную второй и четвертый признаки. Теперь построим модель линейной регрессии на них.

In [6]:
x1 = x.drop([0, 2], axis=1)
x1

,1,3
0,2.308685,2.692872
1,1.303374,3.463632
2,1.824634,1.861594
3,2.673921,1.667461
4,1.973673,1.697552
5,0.777508,4.544106
6,1.812067,1.518746
7,2.269714,2.610812
8,3.175640,5.107734
9,2.879887,1.596360


In [7]:
lr = LinearRegression(fit_intercept=False).fit(x1, y)

Посчитаем предсказанные моделью значения y и ошибку в предсказании модели:

In [8]:
y1 = x1[1]*lr.coef_[0] + x1[3]*lr.coef_[1]
y1

0     9.505510
1     8.397632
2     7.096572
3     8.745344
4     7.183174
5     8.874571
6     6.532917
7     9.287959
8    15.264412
9     9.107731
dtype: float64

In [11]:
e1 = y - y1
e1

0   -0.243576
1    1.302731
2    1.575576
3    0.002625
4   -0.996283
5   -1.341444
6    1.043521
7    3.161695
8   -0.974305
9   -2.424119
dtype: float64

Посчитаем коэффициент детерминации:

In [9]:
def sum_of_squares(samples: np.ndarray) -> float:
    """Сумма квадратов отклонений.
    """
    
    return ((samples - samples.mean()) ** 2).sum()

In [12]:
R = 1 - sum_of_squares(e1) / sum_of_squares(y)
R

0.5717210122042007

__Задача 2__

Является ли значимым уравнение регрессии, полученное в предыдущей задаче? В качестве ответа запишите 1, если является, и 0 иначе.


__Решение__

Чтобы найти ответ проведем тест Фишера. Определим праметры k1 и k2:

In [13]:
n = 10
m = 2

In [14]:
k1 = m
k2 = n - m - 1
k1, k2

(2, 7)

Возьмём уровень значимости $\alpha = 0.05$. Нужный нам квантиль можно посчитать с помощью функции `scipy.stats.f.ppf`:

In [15]:
from scipy import stats

In [16]:
alpha = 0.05
t = stats.f.ppf(1 - alpha, k1, k2)
t

4.73741412777588

Итак, критическая область:
$$\Omega_\alpha = \left( 4.737, \infty \right)$$

Считаем значение статистики:

In [17]:
F = (R / k1) / ((1 - R) / k2)
F

4.672243093253918

Значение статистики не попало в критическую область. Уравнение регрессии считаем значимым.

Ответ: 1

__Задача 3__

Для проведения A/B-тестирования сайта интернет-магазина были получены следующие данные: страница `A` была посещена 2509 раз, из них 77 закончились совершением покупки, страница `B` была посещена 1465 раз, 60 из них закончились совершением покупки. Является ли значимым отличие конверсии на страницах `A` и `B`? В качестве ответа запишите 1, если является, и 0 иначе.

_Подсказка_. Реализуйте двухвыборочный t-тест. В качестве выборок здесь можно взять наборы меток совершения покупки (0 или 1) каждым посетителем.

__Решение__

Проверим гипотезу о том, что математическое ожидание двух выборок A и B равны. Если это так, то отличие конверсий на страницах A и B не является значимым. В качестве выборок берем наборы меток совершения покупки (0 или 1) каждым посетителем. Реализуем двухвыборочный t-тест.

In [27]:
n1 = 2509
n2 = 1465
x1_m = 77 / n1
x2_m = 60 / n2
x1_m, x2_m

(0.03068951773614986, 0.040955631399317405)

In [26]:
s1 = 77*(1-x1_m)**2 / (n1 -1)
s2 = 60*(1-x2_m)**2 / (n2 - 1)
s1, s2

(0.028846226654324757, 0.037695332005929595)

Посчитаем значение  $\sigma_\Delta$:

In [28]:
s_delta = np.sqrt(s1 ** 2 / n1 + s2 ** 2 / n2)
s_delta

0.0011408643964951916

Посчитаем значение статистики: $$t = \dfrac{\overline{X_1} - \overline{X_2}}{\sigma_\Delta}$$

In [29]:
t = (x1_m - x2_m) / s_delta
t

-8.998539786766687

Зафиксируем уровень значимости $\alpha = 0.05$. Теперь нужно посчитать квантили распределения Стьюдента. Посчитаем число степеней свободы этого распределения:

In [30]:
df = (s1 ** 2 / n1 + s2 ** 2 / n2) ** 2 / \
    ((s1 ** 2 / n1) ** 2 / (n1 - 1) + (s2 ** 2 / n2) ** 2 / (n2 - 1))

df

2467.912992699112

Рассчитаем квантили:

In [31]:
alpha = 0.05

t1 = stats.t.ppf(alpha / 2, df=df)
t2 = stats.t.ppf(1 - alpha / 2, df=df)

t1, t2

(-1.960925694023086, 1.9609256940230855)

Итак, критическая область имеет вид:
$$\Omega_\alpha = (-\infty, -1.96) \cup (1.96, \infty)$$

Значение статистики попало в критическую область. Гипотеза о равенстве математических ожиданий отвергается. Значит, утверждение о том, что различие конверсий не является значимым ложно.

Ответ: 1